## 初始化

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
sys.path.append('../')


## Load cfg and initialize

In [3]:
from dotenv import load_dotenv

load_dotenv('../config/.env', override=True)

import os
from modelscope.utils.config import Config

# model_cfg_file = os.getenv('LOCAL_LLM_CONFIG_FILE')
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE') 

# print(model_cfg_file)
print(tool_cfg_file)

# model_cfg = Config.from_file(model_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)



2023-08-30 19:06:43,492 - modelscope - INFO - PyTorch version 2.0.1+cu117 Found.
2023-08-30 19:06:43,497 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-08-30 19:06:43,520 - modelscope - INFO - Loading done! Current index file version is 1.8.4, with md5 19a5f85d75c8eaa96a7eba32b8e082cb and a total number of 902 components indexed


../config/cfg_tool_new.json


In [4]:
from langchain.embeddings import ModelScopeEmbeddings, DashScopeEmbeddings
from langchain.vectorstores import FAISS
from modelscope_agent.retrieve import ToolRetrieval

embeddings = DashScopeEmbeddings(model="text-embedding-v1")
tool_retrieval = ToolRetrieval(embeddings, FAISS) 

## 本地路径

In [5]:
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.llm import QWen, LocalLLM
from modelscope_agent.prompt import QWenPromptGenerator
from modelscope_agent.output_parser import QwenOutputParser

# llm = QWen(model_cfg)


model_name = 'qwen-7b-agent'
model_cfg = {
    'qwen-7b-agent':{
        'model_id': '../model/Qwen-7B-Agent',
        'model_revision': 'v1.0.5',
        'use_raw_generation_config': True,
        'custom_chat': True
    }
}

llm = LocalLLM(model_name, model_cfg)



[2023-08-30 19:06:45,963] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-08-30 19:06:46.368738: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


In [6]:
agent = AgentExecutor(llm, tool_cfg, tool_retrieval=tool_retrieval)

In [7]:
agent.reset()
agent.run("写一个 2023上海世界人工智能大会 20 字以内的口号，并念出来", remote=True)

raw_text: 

<|system|>:

<|user|>:<|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。


当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_speech-generation", "description": "文本转语音服务，将文字转换为自然而逼真的语音，可配置男声/女声", "parameters": [{"name": "input", "description": "要转成语音的文本", "required": true}, {"name": "gender", "description": "用户身份", "required": true}]}

2. {"name": "modelscope_text-translation-en2zh", "description": "根据输入指令，将相应的英文文本翻译成中文回复", "parameters": [{"name": "input", "description": "用户输入的英文文本", "required": true}]}

3. {"name": "modelscope_text-translation-zh2en", "description": "根据输入指令，将相应的中文文本翻译成英文回复", "parameters": [{"name": "input", "description": "用户输入的中文文本", "required": true}]}

4. {"name": "modelscope_image-chat", "description": "图文对话和图像描述服务，针对输入的图片和用

**************************************************round 1**************************************************

AI创造未来，上海引领世界。 

（英文翻译：AI shapes the future, Shanghai leads the world.） 

</s>:

[]

In [8]:
agent.reset()

agent.run("生成一张有花有酒的图片", remote=True)

raw_text: 

<|system|>:

<|user|>:<|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。


当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

2. {"name": "modelscope_image-chat", "description": "图文对话和图像描述服务，针对输入的图片和用户的文本输入，给出文本回复", "parameters": [{"name": "image", "description": "用户输入的图片", "required": true}, {"name": "text", "description": "用户输入的文本", "required": true}]}

3. {"name": "modelscope_video-generation", "description": "视频生成服务，针对英文文本输入，生成一段描述视频；如果是中文输入同时依赖插件modelscope_text-translation-zh2en翻译成英文", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

4. {"name": "modelscope_speech-generati

**************************************************round 1**************************************************

<IPython.core.display.JSON object>

raw_text: 

<|system|>:

<|user|>:<|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。


当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

2. {"name": "modelscope_image-chat", "description": "图文对话和图像描述服务，针对输入的图片和用户的文本输入，给出文本回复", "parameters": [{"name": "image", "description": "用户输入的图片", "required": true}, {"name": "text", "description": "用户输入的文本", "required": true}]}

3. {"name": "modelscope_video-generation", "description": "视频生成服务，针对英文文本输入，生成一段描述视频；如果是中文输入同时依赖插件modelscope_text-translation-zh2en翻译成英文", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

4. {"name": "modelscope_speech-generati

**************************************************round 2**************************************************

</s>

[{'result': ![IMAGEGEN](https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1e5e2015/20230830/1907/f2073309-e43f-406e-b562-44933620cbe6-1.png?Expires=1693480084&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=XSR1Eo5aTyqBooAeOr328sl0AM4%3D)}]

## 从modelscope拉

In [9]:
model_name = 'qwen-7b-agent'
model_cfg = {
    'qwen-7b-agent':{
        'model_id': 'damo/MSAgent-Qwen-7B',
        'model_revision': 'v1.0.0',
        'use_raw_generation_config': True,
        'custom_chat': True
    }
}


llm2 = LocalLLM(model_name, model_cfg)


2023-08-30 19:12:47,372 - modelscope - INFO - Use user-specified model revision: v1.0.0
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


In [10]:
agent2 = AgentExecutor(llm, tool_cfg, tool_retrieval=tool_retrieval)

In [11]:
agent2.reset()

agent2.run("生成一张有花有酒的图片", remote=True)

raw_text: 

<|system|>:

<|user|>:<|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。


当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

2. {"name": "modelscope_image-chat", "description": "图文对话和图像描述服务，针对输入的图片和用户的文本输入，给出文本回复", "parameters": [{"name": "image", "description": "用户输入的图片", "required": true}, {"name": "text", "description": "用户输入的文本", "required": true}]}

3. {"name": "modelscope_video-generation", "description": "视频生成服务，针对英文文本输入，生成一段描述视频；如果是中文输入同时依赖插件modelscope_text-translation-zh2en翻译成英文", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

4. {"name": "modelscope_speech-generati

**************************************************round 1**************************************************

<IPython.core.display.JSON object>

raw_text: 

<|system|>:

<|user|>:<|system|>:你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。


当前对话可以使用的插件信息如下，请自行判断是否需要调用插件来解决当前用户问题。若需要调用插件，则需要将插件调用请求按照json格式给出，必须包含api_name、parameters字段，并在其前后使用<|startofthink|>和<|endofthink|>作为标志。然后你需要根据插件API调用结果生成合理的答复； 若无需调用插件，则直接给出对应回复即可。

1. {"name": "modelscope_image-generation", "description": "图像生成服务，针对文本输入，生成对应的图片", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

2. {"name": "modelscope_image-chat", "description": "图文对话和图像描述服务，针对输入的图片和用户的文本输入，给出文本回复", "parameters": [{"name": "image", "description": "用户输入的图片", "required": true}, {"name": "text", "description": "用户输入的文本", "required": true}]}

3. {"name": "modelscope_video-generation", "description": "视频生成服务，针对英文文本输入，生成一段描述视频；如果是中文输入同时依赖插件modelscope_text-translation-zh2en翻译成英文", "parameters": [{"name": "text", "description": "用户输入的文本信息", "required": true}]}

4. {"name": "modelscope_speech-generati

**************************************************round 2**************************************************

</s>

[{'result': ![IMAGEGEN](https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1e5e2015/20230830/1913/db178fbf-fd65-4629-9cc0-9982d9c95c65-1.png?Expires=1693480425&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=VhR%2BmnBatv7g%2Bx%2FflPNLy78bb8U%3D)}]